# Создание модели предсказания тарифного плана телекоммуникационной компании "Мегалайн"

## Актуальность проекта

### Цель проекта

Построить модель предсказания тарифного плана с максимально большим значением точности (от 0.75 и выше). 

### Описание проекта

В вашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы Смарт и Ультра (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных была выполнена в прошлом проекте.

## Загрузка данных. Изучение общей информации

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Прочитаем файл с данными

In [2]:
url = 'https://drive.google.com/file/d/1roNkPYZbJHK0QfVqWonkwIEALpAi9Hu2/view?usp=sharing'
url_2 = 'https://drive.google.com/uc?id=' + url.split('/')[-2]

In [3]:
df = pd.read_csv(url_2)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В представленном датасете 3214 записей, каждая из которых представляет суммарные данные за месяц по количеству телефонных звонков, общей длительности, количестве отправленных СМС, объем использованного интернет-трафика, и данные о том, какой тарифный план использовал абонент в этом месяце. 

In [5]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
df.is_ultra.value_counts() 

0    2229
1     985
Name: is_ultra, dtype: int64

## Разделение датасета на выборки

Разобьем данные на выборки: _features_ - признаки, то есть все столбца таблицы, за исключением целевого - _is_ultra_ , и _target_ - целевой столбец, тот который мы хотим спрогнозировать. 

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Для создания модели разделим датасет на 3 части:
* тренеровочная выборка - 60%
* тестовая выборка - 20%
* валидационная выборка - 20% 

Сначала мы от всего датасета отделим тестовую выборку. А затем от остатка отделим валидационную выборку. Все что останется - будет обучающей выборкой. При отделении валидационной выборки test_size укажем =0.25, так как отделение валидационной выборки происходит от остатка (после отделения тестовой), и по количеству записей 25% от остатка будут равны 20% от всего датасета.

In [8]:
features_remain, features_test, target_remain, target_test = train_test_split(features, target, test_size=0.2, 
                                                                              stratify = target, random_state=42)
features_train, features_valid, target_train, target_valid = train_test_split(features_remain, target_remain, 
                                                                              test_size=0.25, stratify = target_remain, 
                                                                              random_state=42)

Итог: датасет поделен на обучающую, тестовую и валидационную выборки. В свою очередь каждая из этих выборок поделена на признаки и целевой показатель.

<div class="alert alert-success">

**Комментарий ревьювера**

Разделение выполнено корректно, пропорции выбраны адекватные 👍🏻

## Исследование разных моделей

### Дерево решений

Построим модель на основе алгоритма Дерево решений. Так как гиперпараметры алгоритма могут значительно повлиять на точность модели, используем цикл, чтобы найти оптимальную максимальную глубину дерева решений (в промежутке от 1 до 20), точнее ту глубину, которая даст максимальную точность работы модели.

In [9]:
best_model_tree = None
best_accuracy_t = 0
best_depth_t = 0

In [10]:
for depth in range (1, 20):
    model_tree = DecisionTreeClassifier(random_state=42, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_valid_tree = model_tree.predict(features_valid)
    accuracy_tree = accuracy_score(target_valid, predictions_valid_tree)
    if accuracy_tree > best_accuracy_t:
        best_model_tree = model_tree
        best_accuracy_t = accuracy_tree
        best_depth_t = depth

In [11]:
best_accuracy_t

0.7900466562986003

In [12]:
best_depth_t

5

In [13]:
best_model_tree = DecisionTreeClassifier(random_state=42, max_depth=5)
best_model_tree.fit(features_train, target_train)
best_predictions_valid_tree = best_model_tree.predict(features_valid)

In [14]:
report = classification_report(target_valid, best_predictions_valid_tree, target_names=['1', '0'])
print(report) 

              precision    recall  f1-score   support

           1       0.79      0.94      0.86       446
           0       0.78      0.44      0.56       197

    accuracy                           0.79       643
   macro avg       0.78      0.69      0.71       643
weighted avg       0.79      0.79      0.77       643



В результате работы цикла, была создана наиболее оптимальная модель (котрую сохранили под именем _best_model_tree_), с глубиной дерева =5. Настощая модель способна предсказать с точностью до 79%

### Случайный лес

Построим модель на основе алгоритма Случайный лес. Как и в случае с деревом, в цикле подберем количество деревьев (от 1 до 4) и глубину дерева (от 1 до 10), которые в сочитании дадут наибольшую точность

In [15]:
best_model_forest = None
best_score_f = 0
best_depth_f = 0
best_n_estimator_f = 0

In [16]:
for est in range(1, 5):
    for depth_f in range(1, 11):
        model_forest = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth_f)
        model_forest.fit(features_train, target_train)
        predictions_valid_forest = model_forest.predict(features_valid)
        accuracy_forest = accuracy_score(target_valid, predictions_valid_forest)
        if accuracy_forest > best_score_f:
            best_model_forest = model_forest
            best_score_f = accuracy_forest
            best_depth_f = depth_f
            best_n_estimator_f = est

In [17]:
best_score_f

0.7916018662519441

In [18]:
best_depth_f

5

In [19]:
best_n_estimator_f

2

In [20]:
best_model_forest = RandomForestClassifier(random_state=42, n_estimators=2, max_depth=5)
best_model_forest.fit(features_train, target_train)
best_predictions_valid_forest = best_model_forest.predict(features_valid)

In [21]:
report_f = classification_report(target_valid, best_predictions_valid_forest, target_names=['1', '0'])
print(report_f) 

              precision    recall  f1-score   support

           1       0.79      0.95      0.86       446
           0       0.79      0.44      0.56       197

    accuracy                           0.79       643
   macro avg       0.79      0.69      0.71       643
weighted avg       0.79      0.79      0.77       643



При использовании алгоритма случайный лес, достигнут практически такой же уровень точности. Этот показатель был достигнут при 2 деревьях глубиной = 5.

### Логистическая регрессия

Построим модель на основе логистической регрессии. И посчитаем точность на валидационной выборке

In [22]:
model_logr = LogisticRegression(random_state=42, solver='liblinear')
model_logr.fit(features_train, target_train)
predictions_valid_logr = model_logr.predict(features_valid)
accuracy_logr = accuracy_score(target_valid, predictions_valid_logr)

In [23]:
accuracy_logr

0.7060653188180405

Точность логистической регрессии оказалась на 9% ниже двух остальных алгоритмов, поэтому эту модель дальше проверять не будем.

## Проверьте модель на тестовой выборке

Проверим на тестовой выборке точность работы модели на базе случайного леса

In [24]:
predictions_test_forest = best_model_forest.predict(features_test)

In [25]:
accuracy_test_forest = accuracy_score(target_test, predictions_test_forest)
accuracy_test_forest

0.7993779160186625

Точность случайного леса на тестовой выборке составляет 79.9%, что соответствует точности на основе валидационной выборки.

Проверим дерево решений на тестовой выборке

In [26]:
predictions_test_tree = best_model_tree.predict(features_test)

In [27]:
accuracy_test_tree = accuracy_score(target_test, predictions_test_tree)
accuracy_test_tree

0.8009331259720062

Точность дерева решений на тестовой выборке составила 80%, это на 1% выше, чем при проверке на валидационной выборке. Но показатели точности для случайного леса и дерева решений очень близки. 

## Проверка модели на адекватность

Для того, чтобы вручную проверить адекватно ли модель предлагает тарифные планы, создадим условный датафрейм, куда включим самые разные паттерны клиентского поведения. За основу возьмем условия тарифных плнов, а также выводы прошлого статистического исследования. В новую таблицу_new_features_ будут включены следующие ситуации:
1. данные, равные условиям тарифного плана Ультра
2. превышение условий тарифа Ультра
3. данные, равные условиям тарифного плана Смарт
4. минимальное использование - меньше, чем в среднем на тарифе Смарт
5. промежуток 1 (по результатам статистического исследования была дана рекомендация разработать промежуточные тарифы (2 тарифа между Смарт и Ультра)
6. промежуток 2
7. медианное использование звонков, сообщений, интернет-трафика на тарифе Смарт
8. медианное использование тарифа Ультра

Согласно ожиданиям, модель должна предсказать тариф Ультра в 1,2, и возможно в 5 и последней записях.

In [28]:
new_features = pd.DataFrame(
    [[100, 3000, 1000, 30720], 
     [1000, 3500, 1300, 40000], 
     [30, 500, 50, 15360], 
     [10, 300, 10, 10000], 
     [60, 600, 50, 17408], 
    [70, 700, 50, 20480], 
     [40, 450, 20, 17000], 
     [55, 550, 40, 19500]], 
    columns=features.columns)

In [29]:
new_features

,calls,minutes,messages,mb_used
0,100,3000,1000,30720
1,1000,3500,1300,40000
2,30,500,50,15360
3,10,300,10,10000
4,60,600,50,17408
5,70,700,50,20480
6,40,450,20,17000
7,55,550,40,19500


In [30]:
predictions_new_forest = best_model_forest.predict(new_features)
predictions_new_forest

array([1, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [31]:
predictions_new_tree = best_model_tree.predict(new_features)
predictions_new_tree

array([1, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

Была проверена работа обоих моделей (и лес и дерево). Предсказания модели совпали с нашими ожиданиями. В связи с этим модель можно считать адекватно работающей с точностью 78.8-80%

**Общий вывод**

В рамках данного проекта предоставленный датасет был разделен на 3 выборки: обучающую, валидационную и тестовую. На основании обучающей выборки было построены 3 модели на основе разных алгоритмов: дерево решений, случайный лес, логистическая регрессия. 

Точность логистической регрессии была ниже остальных алгоритмов. Вероятно, это связано с тем, что в обучающей выборке мало признаков - всего 4 признака, при этом дисперсия по каждому конкретному показателю высокая. Поэтому предсказания логистической регрессии не так точны, как в других алгоритмах.

Дерево решений (с максимальной глубиной 7) и случаный лес (с максимально возможным количеством деревьев =4 и максимальной глубиной деева = 8) показали близкие результаты по точности. Поэтомупо результатам проекта я рекомендовала бы использовать модель на основе Дерева решений, так как для обучения модели берется всего 4 признака (относительно небольшое количество), то и загружать технические мощности компании более сложным (затратным) алгоритмом нет оснований